<a href="https://colab.research.google.com/github/HimashiRathnayake/CMCS-Text-Classification/blob/main/XLM-R/CMCS_single_task_learning_with_XLMR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine Tune XLM-R 
Humor Detection & Hate speech Detection of Sinhala-English Code-Mixed Data.

### **Parameters**

**User Parameters**

In [ ]:
technique = "hate speech" #@param ["humor", "hate speech"]
experiment_no = "1" #@param [] {allow-input: true}
over_sampling_technique = "ROS" #@param ["", "ROS","ADASYN", "SMOTE", "BorderlineSMOTE"]
sampling_strategy = "1:0.25:0.25" #@param [] {allow-input: true}
random_state = 45 #@param

if technique == "humor" :
  NO_OUTPUT_LAYERS = 2
  tag_set = ["Humorous", "Non-Humorous"]
elif technique == "hate speech":
  NO_OUTPUT_LAYERS = 3
  tag_set = ["Abusive", "Hate-Inducing", "Not offensive"]

In [ ]:
MAX_LEN = 128
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
EPOCHS = 3

**Folder Paths**

In [ ]:
dataset_path = "/content/drive/Shareddrives/FYP/corpus/çompleted_draft.csv"
model_save_path = "/content/drive/Shareddrives/FYP/Humor_HateSpeech_detection/XLMR/"+technique+"/"+experiment_no

**Dependencies**

In [ ]:
# !pip install transformers
# !pip install sentencepiece
# !pip install datasets
# # !pip install optuna

In [ ]:
import re
import time
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, TrainingArguments, Trainer, AdamW, get_scheduler
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
import io
import seaborn as sns
from datasets import load_metric
from collections import Counter
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE
import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Oversampling**

In [ ]:
def apply_oversampling(x, y):

  (unique, counts) = np.unique(y, axis=0, return_counts=True)
  print("Class Distribution Without Oversampling", counts)

  # define oversampling strategy
  if (over_sampling_technique == ""):
    return x, y
  elif (over_sampling_technique == "ROS"):
    if (technique=="humor"):
      oversample = RandomOverSampler(sampling_strategy = float(sampling_strategy))
    else:
      sampling_ratio = sampling_strategy.split(":");
      oversample = RandomOverSampler(sampling_strategy = {
          0:int(counts[0]*float(sampling_ratio[0])), 
          1:int(counts[0]*float(sampling_ratio[1])), 
          2:int(counts[0]*float(sampling_ratio[2]))
          })
  elif (over_sampling_technique == "ADASYN"):
    oversample = ADASYN(sampling_strategy="minority")
  elif (over_sampling_technique == "SMOTE"):
    oversample = SMOTE()
  elif (over_sampling_technique == "BorderlineSMOTE"):
    oversample = BorderlineSMOTE()

  # fit and apply the transform
  X_over, y_over = oversample.fit_resample(x, y)

  (unique, counts) = np.unique(y_over, axis=0, return_counts=True)
  print("Class Distribution After Oversampling", counts)

  return X_over, y_over

### **Load & Preprocess Dataset**

In [ ]:
def preprocess_texts(sentences):
  sentences = [re.sub(r'http\S+','',s) for s in sentences]
  sentences = [s.replace('#','') for s in sentences]
  sentences = ["[CLS] " + s + " [SEP]" for s in sentences]
  return sentences

In [ ]:
all_data = pd.read_csv(dataset_path)

if (technique == "humor"):
  all_data = all_data[['Sentence', 'Humor']]
elif (technique == "hate speech"):
  all_data = all_data[['Sentence', 'Hate_speech']]
else:
  all_data = all_data[['Sentence', 'Offensive']]

all_data.columns = ['Sentence', 'Label']
all_data['Label'], uniq = pd.factorize(all_data['Label'])

X = all_data['Sentence'].values.tolist()
y = all_data['Label'].values.tolist()

# X = preprocess_texts(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = random_state)

In [ ]:
X_train = np.array(X_train).reshape(-1, 1)
X_train, y_train = apply_oversampling(X_train, y_train)
X_train = [x[0] for x in X_train.tolist()]
# y_train = y_train.tolist()

Class Distribution Without Oversampling [11044   308   814]
Class Distribution After Oversampling [11044  2761  2761]


##### **Preprocess Data**

In [ ]:
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base", do_lower_case=True) #######################################################uncased

In [ ]:
encoded_X_train = tokenizer(X_train, truncation=True, padding=True, max_length=MAX_LEN)
encoded_X_test = tokenizer(X_test, truncation=True, padding=True, max_length=MAX_LEN)

In [ ]:
class DatasetObject(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = DatasetObject(encoded_X_train, y_train)
test_dataset = DatasetObject(encoded_X_test, y_test)

In [ ]:
train_sampler = RandomSampler(train_dataset)
validation_sampler = SequentialSampler(test_dataset)
train_loader = DataLoader(train_dataset, sampler=train_sampler , batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, sampler=validation_sampler , batch_size=BATCH_SIZE) 

### **Fine-Tuning**

#### **Initialize the model**

In [ ]:
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=NO_OUTPUT_LAYERS)
model.cuda()
print("Done")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

Done


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla K80'

In [ ]:
import gc
# del all_data, encoded_X_train, encoded_X_test, X_train, X_test, tokenizer, DatasetObject
torch.cuda.empty_cache()
gc.collect()

521

#### **Fine-tuning in PyTorch with the Trainer API**

##### **Fine-tune & Test the model**

In [ ]:
def compute_metrics(eval_pred):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("f1")
    metric4 = load_metric("accuracy")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = metric1.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]
    macro_precision = metric1.compute(predictions=predictions, references=labels, average="macro")["precision"]
    macro_recall = metric2.compute(predictions=predictions, references=labels, average="macro")["recall"]
    macro_f1 = metric3.compute(predictions=predictions, references=labels, average="macro")["f1"]
    return {"accuracy":accuracy, "precision": precision, "recall": recall, "f1": f1, "macro_precision": macro_precision, "macro_recall": macro_recall, "macro_f1": macro_f1}

In [ ]:
# Default Hyperparameters
# training_args = TrainingArguments("test_trainer") 
# learning_rate=5e-5, batch_size=8,  weight_decay=0, num_train_epochs=3

training_args = TrainingArguments(
    learning_rate = LEARNING_RATE,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    output_dir = model_save_path,
    num_train_epochs = EPOCHS,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
)

In [ ]:
trainer = Trainer(
    model = model, 
    args = training_args, 
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics,
)

**Train the model**

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 16566
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1554


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Macro Precision,Macro Recall,Macro F1
1,0.672000,0.288575,0.886834,0.900408,0.886834,0.893046,0.555893,0.607620,0.577582
2,0.329000,0.293011,0.900148,0.917077,0.900148,0.907097,0.633483,0.698658,0.659958


***** Running Evaluation *****
  Num examples = 1352
  Batch size = 32
Saving model checkpoint to /content/drive/Shareddrives/FYP/Humor_HateSpeech_detection/XLMR/hate speech/1/checkpoint-518
Configuration saved in /content/drive/Shareddrives/FYP/Humor_HateSpeech_detection/XLMR/hate speech/1/checkpoint-518/config.json
Model weights saved in /content/drive/Shareddrives/FYP/Humor_HateSpeech_detection/XLMR/hate speech/1/checkpoint-518/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1352
  Batch size = 32
Saving model checkpoint to /content/drive/Shareddrives/FYP/Humor_HateSpeech_detection/XLMR/hate speech/1/checkpoint-1036
Configuration saved in /content/drive/Shareddrives/FYP/Humor_HateSpeech_detection/XLMR/hate speech/1/checkpoint-1036/config.json
Model weights saved in /content/drive/Shareddrives/FYP/Humor_HateSpeech_detection/XLMR/hate speech/1/checkpoint-1036/pytorch_model.bin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Macro Precision,Macro Recall,Macro F1
1,0.672000,0.288575,0.886834,0.900408,0.886834,0.893046,0.555893,0.607620,0.577582
2,0.329000,0.293011,0.900148,0.917077,0.900148,0.907097,0.633483,0.698658,0.659958
3,0.181500,0.304687,0.916420,0.924205,0.916420,0.919650,0.686671,0.722556,0.701798


***** Running Evaluation *****
  Num examples = 1352
  Batch size = 32
Saving model checkpoint to /content/drive/Shareddrives/FYP/Humor_HateSpeech_detection/XLMR/hate speech/1/checkpoint-1554
Configuration saved in /content/drive/Shareddrives/FYP/Humor_HateSpeech_detection/XLMR/hate speech/1/checkpoint-1554/config.json
Model weights saved in /content/drive/Shareddrives/FYP/Humor_HateSpeech_detection/XLMR/hate speech/1/checkpoint-1554/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1554, training_loss=0.3851877980250649, metrics={'train_runtime': 2455.536, 'train_samples_per_second': 20.239, 'train_steps_per_second': 0.633, 'total_flos': 3269052658561536.0, 'train_loss': 0.3851877980250649, 'epoch': 3.0})

**Test the fine-tuned model**

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1352
  Batch size = 32


{'epoch': 3.0,
 'eval_accuracy': 0.9164201183431953,
 'eval_f1': 0.9196499274261476,
 'eval_loss': 0.30468660593032837,
 'eval_macro_f1': 0.7017978588269406,
 'eval_macro_precision': 0.6866711637625028,
 'eval_macro_recall': 0.722555869988005,
 'eval_precision': 0.9242047251079533,
 'eval_recall': 0.9164201183431953,
 'eval_runtime': 25.0298,
 'eval_samples_per_second': 54.016,
 'eval_steps_per_second': 1.718}

In [ ]:
# predictions = trainer.predict(test_dataset)

In [ ]:
# preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
# test_set = {'Sentence': X_test,
#         'Label':predictions.label_ids,
#         'Prediction':preds}
  
# # Create DataFrame
# df = pd.DataFrame(test_set)

In [ ]:
# df

In [ ]:
model.save_pretrained("/content/drive/Shareddrives/FYP/final_models/xlmr-st/hate")

Configuration saved in /content/drive/Shareddrives/FYP/final_models/xlmr-st/hate/config.json
Model weights saved in /content/drive/Shareddrives/FYP/final_models/xlmr-st/hate/pytorch_model.bin


##### **Hyperparameter Search**

In [ ]:
# args = TrainingArguments(
#     "test-glue",
#     evaluation_strategy = "epoch",
#     save_strategy = "epoch",
#     learning_rate=LEARNING_RATE,
#     per_device_train_batch_size=BATCH_SIZE,
#     per_device_eval_batch_size=BATCH_SIZE,
#     num_train_epochs=EPOCHS,
#     weight_decay=0.01,
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_macro_f1",
# )

In [ ]:
# def my_hp_space(trial):
#     return {
#         # "learning_rate": trial.suggest_float("learning_rate", 1e-7, 1e-3, log=True),
#         "num_train_epochs": trial.suggest_discrete_uniform("num_train_epochs", 1, 5, 1),
#         # "seed": trial.suggest_int("seed", 1, 40),
#         # "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16, 32, 64]),
#     }

In [ ]:
# trainer = Trainer(
#     model_init=model_init,
#     args=args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     # tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

# best_run = trainer.hyperparameter_search(n_trials=5, direction="maximize", hp_space=my_hp_space)

#### **Fine tuning with native PyTorch**

**Fine-tune the model**

In [ ]:
# def epoch_time(start_time, end_time):
#   elapsed_time = end_time - start_time
#   elapsed_mins = int(elapsed_time / 60)
#   elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
#   return elapsed_mins, elapsed_secs

In [ ]:
# # apply different hyperpameters for specific parameter groups
# # param_optimizer = list(model.named_parameters())

# # optimizer_grouped_parameters = [
# #     {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
# #      'weight_decay_rate': 0.01},
# #     {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
# #      'weight_decay_rate': 0.0}
# # ]
# # optimizer = AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE)
# optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

In [ ]:
# train_loss_set = []
# num_training_steps = EPOCHS * len(train_loader)
# progress_bar = tqdm(range(num_training_steps))
# model.train()

# for epoch in range(EPOCHS):
#   start_time = time.time()

#   tr_loss = 0
#   nb_tr_steps = 0
  
#   for batch in train_loader:

#     batch = {k: v.to(device) for k, v in batch.items()}
#     outputs = model(**batch)
#     loss = outputs.loss
#     loss.backward()

#     optimizer.step()
#     optimizer.zero_grad()
#     progress_bar.update(1)

#     train_loss_set.append(loss.item())    
    
#     tr_loss += loss.item()
#     nb_tr_steps += 1

#   end_time = time.time()

#   print(epoch_time(start_time,end_time))
#   print("\nTrain loss: {}".format(tr_loss/nb_tr_steps))

**Validate the model**

In [ ]:
# accuracy = load_metric("accuracy")
# precision = load_metric("precision")
# recall = load_metric("recall")
# f1 = load_metric("f1")
# macro_precision = load_metric("precision")
# macro_recall = load_metric("recall")
# macro_f1 = load_metric("f1")

# model.eval()
# for batch in test_loader:
#     batch = {k: v.to(device) for k, v in batch.items()}
#     with torch.no_grad():
#         outputs = model(**batch)

#     logits = outputs.logits
#     predictions = torch.argmax(logits, dim=-1)
#     accuracy.add_batch(predictions=predictions, references=batch["labels"])
#     precision.add_batch(predictions=predictions, references=batch["labels"])
#     recall.add_batch(predictions=predictions, references=batch["labels"])
#     f1.add_batch(predictions=predictions, references=batch["labels"])
#     macro_precision.add_batch(predictions=predictions, references=batch["labels"])
#     macro_recall.add_batch(predictions=predictions, references=batch["labels"])
#     macro_f1.add_batch(predictions=predictions, references=batch["labels"])

# print(accuracy.compute())
# print(precision.compute(average="weighted"))
# print(recall.compute(average="weighted"))
# print(f1.compute(average="weighted"))
# print("macro averages:")
# print(macro_precision.compute(average="macro"))
# print(macro_recall.compute(average="macro"))
# print(macro_f1.compute(average="macro"))